In [11]:
import os
from scipy import integrate, interpolate
from sympy import exp,besselk,pi,sqrt,Heaviside,sqrt, log, pi
from numpy import logspace, log10
import numpy as np
import sympy as sp
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
import warnings
import pickle
Tmin,Tmax = 1e-15,1e5 #min and max values for evaluating gSTAR

In [2]:
#Load auxiliary (pre-computed) functions:

if not os.path.isfile('gFunctions.pcl'):
    getFunctions('gFunctions.pcl')
 
f = open('gFunctions.pcl','rb')
logger.info("Loading aux functions. Ignoring BSM corrections to g* and g*_S")
gSTAR = pickle.load(f)
gSTARS = pickle.load(f)
Tfunc = pickle.load(f)
f.close()

INFO:__main__:Loading aux functions. Ignoring BSM corrections to g* and g*_S
INFO:pyCode.AuxFuncs:Loading aux functions. Ignoring BSM corrections to g* and g*_S


In [25]:
def getTemperature(x,NS,S0=1.):
    
    xmin = np.log((2*np.pi**2/45.)*gSTARS(Tmin)*Tmin**3)
    xmax = np.log((2*np.pi**2/45.)*gSTARS(Tmax)*Tmax**3)
    xeff = NS + np.log(S0) - 3.*x
        
    if xeff < xmin:  #For T < Tmin, g* is constant
        return ((45./(2*np.pi**2))*np.exp(xeff)/gSTARS(Tmin))**(1./3.)
    elif xeff > xmax: #For T > Tmax, g* is constant
        return ((45./(2*np.pi**2))*np.exp(xeff)/gSTARS(Tmax))**(1./3.)
    else:    
        return float(Tfunc(xeff))
    

class Tval(sp.Function):
    _imp_ = staticmethod(getTemperature)

    def fdiff(self, argindex=2):
        if argindex != 2:
            return 0
        else:
            return self/3

# class dSigmaVdT(sp.Function):
#     _imp_ = staticmethod(lambda T: derivative(sigmaVJan,T,dx=1e-6))

In [27]:
getTemperature(3.,0.)

0.028260479213845172

In [14]:
x,NS = sp.symbols('x,NS')

In [18]:
f = sp.lambdify([x,NS],Tval(x,NS).diff(NS),modules=['numpy','sympy'])

In [19]:
f(3.,0.)

0.00942015973794839